# Demo Cv a .txt. generate_text_files.txt

Este script toma los CV en formato PDF y genera archivos .txt. Se usa la librería **PyMuPdf** https://github.com/pymupdf/PyMuPDF .
Además se procesa para eliminar simbolos innecesarios y no reconocidos. Procesa los archivos .pdf de la carpeta **/resumes** y genera como salida .txt en la carpeta **/resumes_text_output**

In [59]:
%run -i '/home/erwin/Genoma/cv-parser/parser/generate_text_files.py'

### Veamos un ejemplo de salida

In [84]:
file = '/home/erwin/Genoma/cv-parser/resumes_text_output/1586809060281-CV_Constanza_Arredondo_Berrios'
cv_txt = open(file, "r")
print(cv_txt.read())

Constanza Arredondo Berríos 
Ingeniero Comercial, Pontificia Universidad Católica de Chile 
cearredondo@uc.cl 
569 88076057 
Presentación: 
Empática, responsable, alegre y 
creativa. Mi objetivo es 
consolidarme profesionalmente 
en una organización, donde 
pueda aprender y desarrollar 
mis habilidades técnicas y 
sociales. 
Me gusta trabajar en equipo, 
comunicarme, alcanzar 
objetivos y planificar. 
Experiencia: 
Abril 2019  marzo 2020: Working Holiday en Dinamarca. 
Ejecutiva Comercial Banco Bice Part time 
Banco Bice Febrero 2019  Marzo 2019 
Mi trabajo se enfocó en la captación de clientes. Busqué activamente el perfil 
de cliente, trabajé en equipo y desarrollé mis habilidades comunicativas para 
lograr con éxito mis objetivos mensuales. 
Analista Desarrollo Organizacional Fulltime 
Empresas La Polar S.a. Octubre 2018  Diciembre 2018 
Reemplazo de prenatal. Mi principal tarea consistió en dirigir y aplicar las 
capacitaciones de los colaboradores de la empresa. Analicé las bases 

# Demo parser.py
## Puede ser utilizado para filtrar cv para keywords especificas, recuperar mail, nombre, grado más alto, etc.

Primero corremos el script para que procese los archivos ubicados en la carpeta **/resumes**.
Esta funcion genera archivos .JSON

In [61]:
%run -i '/home/erwin/Genoma/cv-parser/parser/parser.py'

[nltk_data] Downloading package punkt to /home/erwin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/erwin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/erwin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Finalizado. Se han procesado 34 CVs


### Veamos un ejemplo parseado

In [103]:
import json


path_to_json = '/home/erwin/Genoma/cv-parser/output_parser/1586809060281-CV_Constanza_Arredondo_Berrios..json'


with open(path_to_json, 'r') as myfile:
    data = myfile.read()

# parse file
cv_parseado = json.loads(data)
pprint.pprint(cv_parseado)

{'Celular': ['56988076057'],
 'Correo': ['cearredondo@uc.cl'],
 'Educacion': ['Universidad Católica de Chile'],
 'Grado': ['Ingeniería'],
 'Lenguajes': ['Inglés avanzado', 'Inglés', 'Español', 'Español nativo'],
 'Linkedin': None,
 'Nombre': 'Constanza Arredondo Berríos ',
 'Nombre archivo': '/home/erwin/Genoma/cv-parser/1586809060281-CV_Constanza_Arredondo_Berrios.',
 'Palabras Claves': ['Alegre',
                     'Creativo',
                     'Adaptable',
                     'Aprender',
                     'Negociación',
                     'Colaboración',
                     'Responsabilidad',
                     'Voluntariado',
                     'Modelamiento',
                     'Amigable',
                     'Sociable'],
 'Skills': ['Excel']}


# Demo seccionar
En principio se seccionan los cv en las siguientes categorias:
1. Extras o Perfil
2. Experiencia
3. Educación
4. Skills

Para poder separar en secciones se ocupa la tabla creada por Daniel, se agregan palabras recurrentes en cada una de las secciones.
Seccionar genera archivos .JSON

### Ejemplo de las primeras 5 filas del archivo que se usa para seccionar

In [63]:
import pandas as pd
categorias = pd.read_csv('/home/erwin/Genoma/cv-parser/parser/seccionesCV.csv', header=1)
categorias.head()

,Perfil,Experiencia,Formación Académica,skills
0,Resumen,Profesional,Educación,Habilidades
1,Biografía,Logros,Formación,Cualificaciones
2,Sobre,Experiencia,Estudios,Conocimientos
3,Acerca de mi,historial,Formación educativa,Habilidades
4,personales,trabajo,académico,tecnologías


In [64]:
%run -i '/home/erwin/Genoma/cv-parser/parser/seccionar.py'

Seccionando CVs: 34
Finalizado


### Veamos un ejemplo seccionado

In [104]:
import pprint
file_seccionado = open('/home/erwin/Genoma/cv-parser/output_seccionado/1586809060281-CV_Constanza_Arredondo_Berrios..json',)
cv_seccionado = json.load(file_seccionado)
pprint.pprint(cv_seccionado)

{'educación': 'ingeniero comercial , pontificia universidad católica chile \n'
              ' cearredondo@uc.cl \n'
              ' capacitación micro empresario santiago , implementación \n'
              ' educación : \n'
              ' pontificia universidad catolica chile \n'
              ' titulada   mar 2013   jul 2017 \n'
              ' tutora par college \n'
              ' uc , 3 año . \n'
              ' south american \n'
              ' business forum \n'
              ' premio a \n'
              ' menor ingresar . \n'
              ' eventos : copenhell , distortion . \n'
              ' ',
 'experiencia': 'empática , responsable , alegrar y \n'
                ' consolidarme profesionalmente \n'
                ' organización , \n'
                ' aprender y desarrollar \n'
                ' comunicarme , alcanzar \n'
                ' objetivo y planificar . \n'
                ' experiencia : \n'
                ' ejecutiva comercial banco bice part timar \n'
   

# Demo ranking
Para la realización del ranking se usa el curriculum seccionado, por lo tanto se cargan los archivos .JSON.

Para el cálculo del ranking se utiliza Tf-idf (frecuencia de término – frecuencia inversa de documento)

\begin{equation*}
idf(t, D) = log \frac{|D|}{|d \in D : t \in D|}
\end{equation*}
Donde D es el número de CV, y el denominador es el número de documentos donde aparece cierta palabra. tf es número de veces que sale una palabra en cierto documento.

\begin{equation*}
\text{Tf-idf} = tf \cdot idf
\end{equation*}

Tf-idf es el producto de dos medidas, frecuencia de término y frecuencia inversa de documento.
El factor de frecuencia inversa de documento que atenúa el peso de los términos que ocurren con mucha frecuencia en la colección de documentos e incrementa el peso de los términos que ocurren pocas veces.

***Algo importante mencionar es que no buscamos la palabra exacta en los CV, sino más bien se calcula la similitud de las palabras, si esta simulitud es mayor a cierto umbral entonces se considera que la palabra fue encontrada.***

## Cálculo de ranking paso a paso:
1. Cargar los archivos .json que contiene a los cvs seccionados
2. Recibir una descripción de cargo.
3. Ampliar la descripción usando cargos parecidos. Usando word embeddings en español (https://github.com/dccuchile/spanish-word-embeddings). Su usa FastText embeddings from SBWC. ***Con este modelo se calculan las similitudes de las palabras y frases.***
3. Recorrer los cvs en sus secciones educación, experiencia y skills. Usando las palabras de la descripción de cargo se calcula para cada uno de los CV la métrica Tf-idf.

Antes de pasar al cálculo de ranking veamos unos ejemplos usando word embeddings en español:

### Definamos algunas funciones útiles

In [71]:
from gensim.models.keyedvectors import KeyedVectors


wordvectors_file_vec ='/home/erwin/Genoma/cv-parser/fasttext-sbwc.3.6.e20.vec'
cantidad = 100000
model = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)




# definamos algunas funciones utiles

def sent2vec(s):
    '''Generate Vectora for sentences.'''
    M = []
    for w in s.split():
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v/np.sqrt((v**2).sum())

def cosine_sim(vec1, vec2):
    '''Return Cosine Similarity.'''
    return  np.dot(vec1,vec2)/(np.linalg.norm(vec1)* np.linalg.norm(vec2))

def get_closest(word, n):
    '''Get n most similar words by words.'''
    #This function can easily be expanded to get similar words to phrases--
    #using sent2vec() method defined in WithWord2Vec notebook. 
    word = word.lower()
    words = [word]
    similar_vals = [1]
    try:
        similar_list = model.most_similar(positive=[word],topn=n)
        
        for tupl in similar_list:
            words.append(tupl[0])
            similar_vals.append(tupl[1])
    except:
        #If word not in vocabulary return same word and 1 similarity-- 
        #see initialisation of words, similarities.
        pass
    
    return words, similar_vals


### Veamos las palabras similiares a "alegre"

In [123]:
#
model.most_similar(positive=['alegre'],topn=6)

[('jovial', 0.6469255685806274),
 ('porto', 0.5816143751144409),
 ('extrovertida', 0.576339066028595),
 ('bonita', 0.5756909847259521),
 ('apacible', 0.5749702453613281),
 ('alegres', 0.574479877948761)]

### Veamos que tanto se parecen 2 palabras, recordar que para ver calcular Tf-idf no buscamos las palabras exactas, nos conformamos con que sea similares. Que tal un umbral de 0.5?

In [115]:
model.similarity('programador', 'desarrollador')

0.7731334

In [121]:
model.similarity('python', 'médico')

0.11089054

### Esta función no esta siendo usada, pero se pretende incorporarla para usar más contexto

In [110]:
frase_1 = 'arquitecto'
frase_2 = 'dibujante técnico'


vector_sentence_1 = sent2vec(frase_1)
vector_sentence_2 = sent2vec(frase_2)
similitud = cosine_sim(vector_sentence_1,vector_sentence_2)

print(similitud)

0.5613615


In [122]:
frase_1 = 'médico cirujano'
frase_2 = 'diseñador gráfico'


vector_sentence_1 = sent2vec(frase_1)
vector_sentence_2 = sent2vec(frase_2)
similitud = cosine_sim(vector_sentence_1,vector_sentence_2)

print(similitud)

0.35082492


### Tomemos una descripción de cargo. La expandimos usando 2 palabras parecidas.

In [116]:
prc_description = '''ingeniería máster python javascript excel desarrollo  experiencia manejo 
machine learning metodologías ágiles liderar equipos planificar organizar trello electrónica carismástico
disruptivo'''







# Expandir descripcion
word_value = {}
similar_words_needed = 2
for word in prc_description.split():
    similar_words, similarity = get_closest(word, similar_words_needed)
    for i in range(len(similar_words)):
        word_value[similar_words[i]] = word_value.get(similar_words[i], 0)+similarity[i]
print(word_value.keys())

dict_keys(['ingeniería', 'ingenierías', 'mecatrónica', 'máster', 'másteres', 'licenciatura', 'python', 'perl', 'scripts', 'javascript', 'script', 'navegador', 'excel', 'powerpoint', 'word', 'desarrollo', 'sostenible', 'socioeconómico', 'experiencia', 'experiencias', 'pericia', 'manejo', 'manejar', 'manejan', 'machine', 'machines', 'rage', 'learning', 'learn', 'education', 'metodologías', 'metodología', 'metodológicas', 'ágiles', 'ágil', 'veloces', 'liderar', 'encabezar', 'lidera', 'equipos', 'equipo', 'clubes', 'planificar', 'planear', 'planificarse', 'organizar', 'organizarlo', 'trello', 'electrónica', 'electrónico', 'electrónicas', 'carismástico', 'disruptivo'])


### Finalmente calculemos el ranking!

In [76]:
# /home/erwin/Genoma/cv-parser/parser/ranking.py

%run -i '/home/erwin/Genoma/cv-parser/parser/ranking.py'

[(64.06678438300953, '1581023751928-CV'),
 (14.96935157846756, '1581000554896-Nabib_M_Garcia_Hirsh_CV'),
 (14.159443311329925, '1588283816605-CV_Erika_Sotomayor_Ulloa'),
 (13.443749454096587, '1580320654107-CV_Gabriel_Garcia_Lorca'),
 (13.198663508227062, '1580573732905-DiegoAlbertoGutierrezBejarano'),
 (13.055698922624781, '1581008153040-CV_Jose_Chávez'),
 (12.435012659850486, 'prueba_genoma_experiencia'),
 (11.32511418059513, '1580913938357-CV_Gianfranco_Vela_ESP_Ene.2020'),
 (11.200322658831475, '1581295975312-CV_Renzo_Salvatierra_R'),
 (10.935572582515752, '1580920711029-Kevin_SQ'),
 (10.711771880025042, '1581631090164-CV_ROSSANA_ALAYZA_-_FEBRERO_2020'),
 (10.488366254374945, '1580497335052-Jose_Medina_13-1-2020'),
 (8.611405855192073, '1581690552715-CV_Gustavo_Martinez_Raya'),
 (8.416487664771893, '1583336373286-CV_Gf_Rosales_2020'),
 (8.278595248820872, '1586786838778-CV_Postulaciones'),
 (7.595220569596056, '1581954259745-CV_2020_ANTONIO_HARTMANN_R'),
 (7.326224459147038, '15928

### Veamos el CV del ganador

In [117]:
file_ra = '/home/erwin/Genoma/cv-parser/resumes_text_output/1581023751928-CV'
cv_txt = open(file_ra, "r")
print(cv_txt.read())

Contactar 
56971920517 Mobile 
chris.hevia@gmail.com 
www.linkedin.com/in/christian 
heviab3902b31 LinkedIn 
Aptitudes principales 
Java 
Gestión de proyectos 
Negociación 
HonorsAwards 
1º Lugar 4to medio 
Mejor de la Carrera Informática 
Análista Programador 
Mejor de la Carrera Informática 
Ingeniería Informática 
Christian Hevia 
Director de Operaciones en Icarus Solutions 
Chile 
Extracto 
Profesional Titulado, con distinción máxima, con 6 años de 
experiencia en Jefatura de Proyectos. 
Capacitado con conocimientos en diversas áreas de la informática, 
principalmente orientado al desarrollo de software, aplicaciones de 
alto rendimiento con énfasis en la calidad del software, liderazgo 
de equipos, alto nivel de análisis y gran motivación para enfrentar 
diversos desafíos. He trabajado en importantes proyectos con 
diversas tecnologías, dentro de las cuales destacan J2ee, Jpa, 
Asp, JavaScript, Xhtml, Metodologías Ágiles, entre otras. 
Sólidos conocimientos en Gestión, Planificaci

### Veamos el peor

In [3]:
file_ra = '/home/erwin/Genoma/cv-parser/resumes_text_output/1581878742461-Curriculum_Joselyn_Mari_2020_1'
cv_txt = open(file_ra, "r")
print(cv_txt.read())

Joselyn Mari Wendorff 
Dni: 72789167. 20 de Octubre de 1993. 
Calle Francisca Zubiaga 212 dpto 501  San Miguel 
Teléfonos: 961606774 
joselynmwendorff@gmail.com 
Egresada en Aviación Comercial y actual estudiante de Marketing en Isil. Soy 
una persona con alta disposición para el trabajo, proactiva con capacidad para 
relacionarse y establecer buena comunicación así como también, trabajar bajo 
presión 
y 
en 
equipo. 
Con 
alto 
sentido 
de 
responsabilidad. 
Mi objetivo y meta profesional es formar parte de una Empresa sólida, en la 
que pueda poner en práctica los conocimientos y habilidades adquiridas. 
Educación 
Instituto San Ignacio De Loyola 
Marketing 
Iii ciclo 
Avia 
Aviación Comercial 
Especialización en Counter Avanzado 
Británico 
Ingles 
Básico  Intermedio 
Colegio Nuestra Señora De La Asuncion 
Secundaria 
Experiencia Laboral 
Bumeran.com 
Ejecutiva de ventas nuevas octubre 2019  Actualidad 
 
Mantenimiento de cartera asignada. 
 
Fidelización de clientes. 
 
Crecimient